In [1]:
import os

In [2]:
%pwd

'c:\\End To End AI Project\\End to end diabetes prediction project\\End-to-End-diabetes-prediction-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\End To End AI Project\\End to end diabetes prediction project\\End-to-End-diabetes-prediction-project'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    kernel: str
    target_column: str

In [9]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.svm
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            kernel=params.kernel,
            target_column = schema.name
            
        )

        return model_trainer_config

In [26]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import pandas as pd
import joblib
import os
from mlProject import logger
import numpy as np


In [29]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config
        
    def train(self):
        data=pd.read_csv(self.config.train_data_path)
        x=data.drop([self.config.target_column],axis=1)
        y=data[[self.config.target_column]]
        scaler=StandardScaler()
        x=scaler.fit_transform(x)
        model=svm.SVC(kernel=self.config.kernel)
        model.fit(np.array(x),np.array(y))

        joblib.dump(model,os.path.join(self.config.root_dir,self.config.model_name))

In [30]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-09-02 10:17:51,425: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-02 10:17:51,430: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-02 10:17:51,438: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-02 10:17:51,442: INFO: common: created directory at: artifacts]
[2024-09-02 10:17:51,445: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\K Tech\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
